In [31]:
import random
from queue import Queue, PriorityQueue
import numpy as np

### Control Plane

In [32]:
SERVICE = 10.0  # SERVICE is the average service time; service rate = 1/SERVICE
ARRIVAL = 3.0  # ARRIVAL is the average inter-arrival time; arrival rate = 1/ARRIVAL
LOAD = SERVICE / ARRIVAL  # This relationship holds for M/M/1
BUFFER_SIZE = 100  # Infinite buffer -> 0
m_SERVERS = 1

TYPE1 = 1

SIM_TIME = 500000

### To take the measurements

In [33]:
class Measure:
    def __init__(self, Narr, Ndep, Nlos, NAverageUser, OldTimeEvent, AverageDelay):
        self.arr = Narr
        self.dep = Ndep
        self.los = Nlos
        self.ut = NAverageUser
        self.oldT = OldTimeEvent
        self.delay = AverageDelay

### Client

In [34]:
class Client:
    def __init__(self, type, arrival_time):
        self.type = type
        self.arrival_time = arrival_time

### Server

In [35]:
class Server(object):
    # constructor
    def __init__(self, service_time):
        self.idle = True  # whether the server is idle or not
        self.service_time = service_time  # service period specific for this server

In [36]:
class MMmB:
    def __init__(self, service_time, servers_count=1, buffer_size=0):
        self.buffer_size = buffer_size  # B
        self.service_time = service_time  # mu
        self._queue: list[Client] = []
        self._servers_count = servers_count
        self._servers_working = 0

    def servers_count(self): return self._servers_count

    def servers_working(self): return self._servers_working

    def queue_size(self): return len(self._queue)

    def is_full(self): return self.queue_size() == self.buffer_size > 0
    
    def servers_busy(self): return self._servers_working == self._servers_count

    def insert(self, event: Client):
        if self.is_full():
            return False
        self._queue.append(event)
        self._servers_working += 1
        return True

    def consume(self):
        if self.servers_working() <= 0:
            return None
        self._servers_working -= 1
        return self._queue.pop()

### Variables

In [37]:
arrivals = 0
users = 0

MMm = MMmB(SERVICE, m_SERVERS, BUFFER_SIZE)

### Arrivals

In [38]:
def arrival(time, FES, queue: MMmB):
    global users

    # print("Arrival no. ",data.arr+1," at time ",time," with ",users," users" )

    # loss detection
    loss = queue.is_full()

    # cumulate statistics
    data.arr += 1
    data.ut += users * (time - data.oldT)  # average users per time unit
    data.oldT = time
    if loss:
        data.los += 1
    else:
        users += 1
        # create a record for the client
        client = Client(TYPE1, time)
        # insert the record in the queue
        queue.insert(client)

    # sample the time until the next event
    inter_arrival = random.expovariate(lambd=1.0 / ARRIVAL)

    # schedule the next arrival
    FES.put((time + inter_arrival, "arrival"))

    # if the server is idle start the service
    if not MMm.servers_busy():
        # sample the service time
        service_time = random.expovariate(1.0 / MMm.service_time)
        # service_time = 1 + random.uniform(0, SEVICE_TIME)

        # schedule when the client will finish the service
        FES.put((time + service_time, "departure"))

### Departures

In [39]:
def departure(time, FES, queue: MMmB):
    global users

    # print("Departure no. ",data.dep+1," at time ",time," with ",users," users" )

    # get the first element from the queue
    client = queue.consume()

    # cumulate statistics
    data.dep += 1
    data.ut += users * (time - data.oldT)
    data.oldT = time

    # do whatever we need to do when clients go away

    data.delay += (time - client.arrival_time)
    users -= 1

    # see whether there are more clients to in the line
    if users > 0:
        # sample the service time
        service_time = random.expovariate(1.0 / SERVICE)

        # schedule when the client will finish the service
        FES.put((time + service_time, "departure"))

## Main

In [40]:
random.seed(42)

data = Measure(0, 0, 0, 0, 0, 0)

# the simulation time 
time = 0

# the list of events in the form: (time, type)
FES = PriorityQueue()

# schedule the first arrival at t=0
FES.put((0, "arrival"))

# simulate until the simulated time reaches a constant
while time < SIM_TIME:
    (time, event_type) = FES.get()

    if event_type == "arrival":
        arrival(time, FES, MMm)

    elif event_type == "departure":
        departure(time, FES, MMm)

# print output data
print("MEASUREMENTS \n\nNo. of users in the queue:", users, "\nNo. of arrivals =",
      data.arr, "- No. of departures =", data.dep)

print("Load: ", SERVICE / ARRIVAL)
print("\nArrival rate: ", data.arr / time, " - Departure rate: ", data.dep / time)
print("Loss rate: ", data.los / time, " - Packets loss: ", data.los)

print("\nAverage number of users: ", data.ut / time)

print("Average delay: ", data.delay / data.dep)
print("Actual queue size: ", len(MMm.queue_size()))

if MMm.queue_size() > 0:
    print("Arrival time of the last element in the queue:", MMm.queue[-1].arrival_time)


4
3
4
4
3
2
3
2
2
1
0


AttributeError: 'NoneType' object has no attribute 'arrival_time'